<a href="https://colab.research.google.com/github/pranavkumar02/Tourist-Flow-And-Seasonality-Analyzer-With-Event-Impact/blob/pranavkumar02-patch-2/Models_capstone.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import warnings
warnings.filterwarnings("ignore")
import random
import numpy as np
import pandas as pd
from datetime import timedelta
from pathlib import Path
from sklearn.metrics import mean_absolute_error, mean_squared_error
from tqdm import tqdm

In [ ]:
# reproducibility
SEED = 42
random.seed(SEED)
np.random.seed(SEED)

In [ ]:
# Config
DATA_PATH = "/content/drive/MyDrive/final_dataset.csv"
OUTPUT_DIR = Path("./forecast_pipeline_output")
FORECASTS_DIR = OUTPUT_DIR / "forecasts"
PLOTS_DIR = OUTPUT_DIR / "plots"
RESULTS_CSV = OUTPUT_DIR / "model_selection_results.csv"

HORIZON = 12  # months to forecast / test horizon
MIN_HISTORY_MONTHS = 36  # minimum months of history to attempt seasonal models
LSTM_MIN_MONTHS = 60  # require more history for LSTM
LSTM_EPOCHS = 100
LSTM_BATCH = 16
LSTM_LOOKBACK = 12  # months window for LSTM
USE_LOG_TRANSFORM = False  # set True to use log1p transform (remember inverse when evaluating)

In [ ]:
# Which models to run (set to False to skip heavy models)
RUN_ETS = True
RUN_PROPHET = True
RUN_SARIMA = True
RUN_LSTM = False

In [ ]:
# Create dirs
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)
FORECASTS_DIR.mkdir(parents=True, exist_ok=True)
PLOTS_DIR.mkdir(parents=True, exist_ok=True)

In [ ]:
# Load dataset
df = pd.read_csv(DATA_PATH)

In [ ]:
# Convert YearMonth to datetime if exists
if "YearMonth" in df.columns:
    df["YearMonth"] = pd.to_datetime(df["YearMonth"], errors="coerce")
else:
    df["YearMonth"] = pd.to_datetime(df[["Year", "Month"]].assign(DAY=1))
# Keep relevant columns
assert "Park" in df.columns, "Dataset must include 'Park' column"
assert "Recreation Visits" in df.columns, "Dataset must include 'Recreation Visits' column"

df = df[["Park", "YearMonth", "Recreation Visits"]].copy()
df = df.dropna(subset=["YearMonth"])
df = df.sort_values(["Park", "YearMonth"])

In [ ]:
# Helper metric
def evaluate_forecast(y_true, y_pred):
    # y_true and y_pred are pandas Series aligned by index
    # if any naive mismatches, reindex
    y_true, y_pred = y_true.align(y_pred, join="inner")
    mae = mean_absolute_error(y_true, y_pred)
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    return mae, rmse

In [ ]:
# Try imports for models
# ETS
from statsmodels.tsa.holtwinters import ExponentialSmoothing

# Prophet
try:
    try:
        from prophet import Prophet
    except Exception:
        # older installation name
        from fbprophet import Prophet
    PROPHET_AVAILABLE = True
except Exception:
    PROPHET_AVAILABLE = False
    RUN_PROPHET = False

# pmdarima (auto_arima)
try:
    import pmdarima as pm
    PM_AVAILABLE = True
except Exception:
    PM_AVAILABLE = False
    RUN_SARIMA = False

# LSTM
try:
    import tensorflow as tf
    from tensorflow.keras.models import Sequential
    from tensorflow.keras.layers import LSTM, Dense, Dropout
    from sklearn.preprocessing import MinMaxScaler
    TF_AVAILABLE = True
except Exception:
    TF_AVAILABLE = False
    RUN_LSTM = False

In [ ]:
# Utility: safe filename
def safe_name(name):
    return "".join(c if c.isalnum() or c in "_- " else "_" for c in name).replace(" ", "_")[:120]

In [ ]:
# Function: prepare series per park
def prepare_series(park_df, asfreq="MS", fill_method="ffill"):
    s = park_df.set_index("YearMonth")["Recreation Visits"].sort_index()
    s = s.asfreq(asfreq)  # missing months become NaN
    # If there's leading NaNs, forward/backfill won't fill them - keep them as NaN
    # Fill small gaps by forward fill, but leave series with too many missing as is
    if fill_method == "ffill":
        s = s.fillna(method="ffill")
    elif fill_method == "interpolate":
        s = s.interpolate(limit_direction="both")
    return s

In [ ]:
# ETS model fit & forecast
def fit_forecast_ets(train, horizon, seasonal_periods=12, seasonal="add", trend="add"):
    # train: pd.Series with datetime index
    model = ExponentialSmoothing(train, trend=trend, seasonal=seasonal, seasonal_periods=seasonal_periods)
    fitted = model.fit(optimized=True, use_boxcox=False, remove_bias=False)
    fc = fitted.forecast(horizon)
    return fitted, fc

# Prophet fit & forecast
def fit_forecast_prophet(train, horizon):
    # train: pd.Series with datetime index, monthly
    # Prophet expects dataframe with ds, y
    dfp = train.reset_index().rename(columns={"YearMonth": "ds", "Recreation Visits": "y"})
    m = Prophet(yearly_seasonality=True, weekly_seasonality=False, daily_seasonality=False)
    m.fit(dfp)
    future = m.make_future_dataframe(periods=horizon, freq="MS")
    fc = m.predict(future)
    # return last horizon forecasts aligned to index
    fc_series = pd.Series(fc["yhat"].values[-horizon:], index=pd.date_range(start=train.index[-1] + pd.offsets.MonthBegin(1), periods=horizon, freq="MS"))
    return m, fc_series

# SARIMA: use pmdarima.auto_arima if available
def fit_forecast_sarima(train, horizon, seasonal_periods=12):
    # train: series
    model = pm.auto_arima(train, seasonal=True, m=seasonal_periods, error_action="ignore", suppress_warnings=True, stepwise=True)
    # get forecast
    fc, conf_int = model.predict(n_periods=horizon, return_conf_int=True, alpha=0.05)
    idx = pd.date_range(start=train.index[-1] + pd.offsets.MonthBegin(1), periods=horizon, freq="MS")
    fc_series = pd.Series(fc, index=idx)
    return model, fc_series

# LSTM: simple window-to-one model (one-step ahead repeated to produce multi-step by recursive forecasting)
def create_lstm_model(input_shape):
    model = Sequential()
    model.add(LSTM(64, input_shape=input_shape, activation="tanh"))
    model.add(Dropout(0.2))
    model.add(Dense(1))
    model.compile(optimizer="adam", loss="mse")
    return model


In [ ]:
def prepare_supervised(series, lookback=12):
    """Return X, y arrays for one-step forecasting using lookback."""
    X, y = [], []
    for i in range(len(series) - lookback):
        X.append(series[i:i + lookback])
        y.append(series[i + lookback])
    return np.array(X), np.array(y)

def fit_forecast_lstm(train, horizon, lookback=12, epochs=50, batch_size=16):
    """
    We'll use recursive multi-step: predict 1-step ahead, append, and repeat.
    train: pd.Series
    """
    scaler = MinMaxScaler()
    train_vals = train.values.reshape(-1, 1)
    scaled = scaler.fit_transform(train_vals).flatten()
    if len(scaled) <= lookback:
        raise ValueError("not enough data for LSTM with lookback")
    X, y = prepare_supervised(scaled, lookback)
    # reshape for LSTM [samples, timesteps, features]
    X = X.reshape((X.shape[0], X.shape[1], 1))
    # simple train/val split
    split = int(0.9 * len(X))
    X_train, y_train = X[:split], y[:split]
    X_val, y_val = X[split:], y[split:]
    model = create_lstm_model((lookback, 1))
    # callbacks
    es = tf.keras.callbacks.EarlyStopping(monitor="val_loss", patience=10, restore_best_weights=True, verbose=0)
    model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=epochs, batch_size=batch_size, callbacks=[es], verbose=0)
    # recursive forecast
    last_window = scaled[-lookback:].tolist()
    preds = []
    for _ in range(horizon):
        x_input = np.array(last_window[-lookback:]).reshape((1, lookback, 1))
        pred_scaled = model.predict(x_input, verbose=0)[0, 0]
        preds.append(pred_scaled)
        last_window.append(pred_scaled)
    preds = np.array(preds).reshape(-1, 1)
    preds_inv = scaler.inverse_transform(preds).flatten()
    idx = pd.date_range(start=train.index[-1] + pd.offsets.MonthBegin(1), periods=horizon, freq="MS")
    fc_series = pd.Series(preds_inv, index=idx)
    return model, fc_series

In [ ]:
# MAIN LOOP: per park
parks = df["Park"].unique()
results = []
failed_parks = []

for park in tqdm(parks, desc="Parks"):
    park_df = df[df["Park"] == park][["YearMonth", "Recreation Visits"]].copy()
    series = prepare_series(park_df, asfreq="MS", fill_method="ffill")
    # drop if too short overall
    if series.dropna().shape[0] < MIN_HISTORY_MONTHS:
        # still try very simple ETS if at least 12 months? else skip
        if series.dropna().shape[0] < 12:
            failed_parks.append((park, "insufficient_history"))
            continue

    # ensure no negative values
    series = series.clip(lower=0)

    # Optionally log transform
    if USE_LOG_TRANSFORM:
        series_trans = np.log1p(series)
    else:
        series_trans = series.copy()

    # drop NaNs at end/start to allow train/test split
    series_trans = series_trans.dropna()
    if len(series_trans) < (HORIZON + 1):
        failed_parks.append((park, "insufficient_after_dropna"))
        continue

    # split
    train = series_trans.iloc[:-HORIZON]
    test = series_trans.iloc[-HORIZON:]

    park_results = {"Park": park, "n_obs": len(series_trans)}
    model_forecasts = {}
    model_metrics = {}

    # ETS
    if RUN_ETS:
        try:
            # choose seasonal type heuristically: multiplicative if coefficient of variation increases with level - optional
            ets_model, ets_fc = fit_forecast_ets(train, HORIZON, seasonal_periods=12, seasonal="add", trend="add")
            # invert transform if used
            if USE_LOG_TRANSFORM:
                ets_fc_inv = np.expm1(ets_fc)
                test_inv = np.expm1(test)
            else:
                ets_fc_inv = ets_fc
                test_inv = test
            mae, rmse = evaluate_forecast(test_inv, ets_fc_inv)
            model_forecasts["ETS"] = ets_fc_inv
            model_metrics["ETS"] = {"MAE": mae, "RMSE": rmse}
        except Exception as e:
            model_metrics["ETS"] = {"error": str(e)}

    # Prophet
    if RUN_PROPHET and PROPHET_AVAILABLE:
        try:
            # Prophet wants original scale, so handle transforms properly
            if USE_LOG_TRANSFORM:
                train_prop = np.expm1(train)
            else:
                train_prop = train
            prophet_model, prophet_fc = fit_forecast_prophet(train_prop, HORIZON)
            if USE_LOG_TRANSFORM:
                prophet_fc_used = prophet_fc  # prophet used inv transform earlier
                test_inv = np.expm1(test)
            else:
                prophet_fc_used = prophet_fc
                test_inv = test
            mae, rmse = evaluate_forecast(test_inv, prophet_fc_used)
            model_forecasts["Prophet"] = prophet_fc_used
            model_metrics["Prophet"] = {"MAE": mae, "RMSE": rmse}
        except Exception as e:
            model_metrics["Prophet"] = {"error": str(e)}

    # SARIMA
    if RUN_SARIMA and PM_AVAILABLE:
        try:
            # pmdarima expects numpy array without index
            if USE_LOG_TRANSFORM:
                sar_train = np.expm1(train)
            else:
                sar_train = train
            sar_model, sar_fc = fit_forecast_sarima(sar_train, HORIZON, seasonal_periods=12)
            if USE_LOG_TRANSFORM:
                test_inv = np.expm1(test)
            else:
                test_inv = test
            mae, rmse = evaluate_forecast(test_inv, sar_fc)
            model_forecasts["SARIMA"] = sar_fc
            model_metrics["SARIMA"] = {"MAE": mae, "RMSE": rmse}
        except Exception as e:
            model_metrics["SARIMA"] = {"error": str(e)}

    # LSTM
    if RUN_LSTM and TF_AVAILABLE and len(series_trans.dropna()) >= LSTM_MIN_MONTHS:
        try:
            # LSTM works on original scale
            lstm_train = np.expm1(train) if USE_LOG_TRANSFORM else train
            lstm_model, lstm_fc = fit_forecast_lstm(lstm_train, HORIZON, lookback=LSTM_LOOKBACK, epochs=LSTM_EPOCHS, batch_size=LSTM_BATCH)
            if USE_LOG_TRANSFORM:
                test_inv = np.expm1(test)
            else:
                test_inv = test
            mae, rmse = evaluate_forecast(test_inv, lstm_fc)
            model_forecasts["LSTM"] = lstm_fc
            model_metrics["LSTM"] = {"MAE": mae, "RMSE": rmse}
        except Exception as e:
            model_metrics["LSTM"] = {"error": str(e)}
    else:
        if RUN_LSTM and not TF_AVAILABLE:
            model_metrics["LSTM"] = {"error": "tensorflow_not_available"}
        elif RUN_LSTM and len(series_trans.dropna()) < LSTM_MIN_MONTHS:
            model_metrics["LSTM"] = {"error": "insufficient_history_for_lstm"}

    # Select best model by RMSE (lowest). models that raised errors are ignored
    valid_models = {m: v for m, v in model_metrics.items() if "RMSE" in v}
    if len(valid_models) == 0:
        # No model successfully fit for this park
        failed_parks.append((park, "no_models_succeeded", model_metrics))
        # Add the park_results with model_metrics for debugging, but don't try to sort by RMSE later for this entry
        results.append({**park_results, "model_metrics": model_metrics})
        continue

    # choose best
    best_model_name = min(valid_models.keys(), key=lambda m: valid_models[m]["RMSE"])
    best_metrics = valid_models[best_model_name]
    park_results.update({
        "best_model": best_model_name,
        "best_MAE": best_metrics["MAE"],
        "best_RMSE": best_metrics["RMSE"],
    })

    # Refit best model to full history (train + test) and produce final forecast for horizon
    try:
        full_series = series_trans.copy()
        if best_model_name == "ETS":
            if USE_LOG_TRANSFORM:
                full_for_fit = np.log1p(full_series)
            else:
                full_for_fit = full_series
            fitted_full, fc_full = fit_forecast_ets(full_for_fit, HORIZON, seasonal_periods=12, seasonal="add", trend="add")
            if USE_LOG_TRANSFORM:
                fc_full = np.expm1(fc_full)
        elif best_model_name == "Prophet":
            # Prophet expects original scale
            full_for_fit = np.expm1(full_series) if USE_LOG_TRANSFORM else full_series
            m_full = Prophet(yearly_seasonality=True, weekly_seasonality=False, daily_seasonality=False)
            dfp = full_for_fit.reset_index().rename(columns={"YearMonth": "ds", "Recreation Visits": "y"})
            m_full.fit(dfp)
            future = m_full.make_future_dataframe(periods=HORIZON, freq="MS")
            fc = m_full.predict(future)
            fc_full = pd.Series(fc["yhat"].values[-HORIZON:], index=pd.date_range(start=full_series.index[-1] + pd.offsets.MonthBegin(1), periods=HORIZON, freq="MS"))
        elif best_model_name == "SARIMA":
            full_for_fit = np.expm1(full_series) if USE_LOG_TRANSFORM else full_series
            sar_full, fc_full = fit_forecast_sarima(full_for_fit, HORIZON, seasonal_periods=12)
        elif best_model_name == "LSTM":
            full_for_fit = np.expm1(full_series) if USE_LOG_TRANSFORM else full_series
            lstm_full_model, fc_full = fit_forecast_lstm(full_for_fit, HORIZON, lookback=LSTM_LOOKBACK, epochs=LSTM_EPOCHS, batch_size=LSTM_BATCH)
        else:
            raise ValueError("Unknown best model")
    except Exception as e:
        park_results.update({"refit_error": str(e)})
        # still save the test metrics as final
        results.append({**park_results, **{"model_metrics": model_metrics}})
        continue

    # Save final forecast to CSV
    fc_df = pd.DataFrame({
        "YearMonth": fc_full.index,
        "Forecast": fc_full.values
    })
    fc_path = FORECASTS_DIR / f"{safe_name(park)}.csv"
    fc_df.to_csv(fc_path, index=False)

    # Optionally save a small actual vs predicted plot for the test horizon
    try:
        import matplotlib.pyplot as plt
        plt.figure(figsize=(8, 4))
        # plot last 36 months actual
        window = 36
        recent_index = series.index[-window:]
        plt.plot(series.loc[recent_index], label="Actual")
        # plot the test predicted from best model (from earlier stored model_forecasts)
        if best_model_name in model_forecasts:
            plt.plot(model_forecasts[best_model_name], label=f"Predicted ({best_model_name}) - test")
        # plot the final forecast (fc_full)
        plt.plot(fc_full, "--", label=f"Forecast ({best_model_name})")
        plt.title(f"{park} — Best: {best_model_name}")
        plt.legend()
        plt.tight_layout()
        plt.savefig(PLOTS_DIR / f"{safe_name(park)}.png")
        plt.close()
    except Exception:
        pass

    # store results
    park_results["model_metrics"] = model_metrics
    park_results["forecast_csv"] = str(fc_path)
    results.append(park_results)

# Save summary results to CSV (flatten)
summary_rows = []
for r in results:
    row = {
        "Park": r["Park"],
        "n_obs": r.get("n_obs", None),
        "best_model": r.get("best_model", None),
        "best_MAE": r.get("best_MAE", None),
        "best_RMSE": r.get("best_RMSE", None), # Use .get() with default None
        "forecast_csv": r.get("forecast_csv", None),
    }
    summary_rows.append(row)

# Filter out rows where best_RMSE is None before sorting
summary_df = pd.DataFrame(summary_rows)
summary_df_sortable = summary_df.dropna(subset=["best_RMSE"])

if len(summary_df_sortable) > 0:
    summary_df_sortable = summary_df_sortable.sort_values("best_RMSE")

# Concatenate the sortable and non-sortable rows for the final summary
summary_df = pd.concat([summary_df_sortable, summary_df[summary_df["best_RMSE"].isna()]])

summary_df.to_csv(RESULTS_CSV, index=False)

# Report
print("Done.")
print(f"Output directory: {OUTPUT_DIR.resolve()}")
print(f"Model selection summary: {RESULTS_CSV}")
print(f"Forecasts directory: {FORECASTS_DIR}")
print(f"Plots directory: {PLOTS_DIR}")
print(f"Failed parks count: {len(failed_parks)} (details in memory)")

# Optionally, display top 10 parks and which model was best
if len(summary_df_sortable) > 0:
    print("\nTop 10 parks by lowest RMSE (best-fit):")
    print(summary_df_sortable[["Park", "best_model", "best_MAE", "best_RMSE"]].head(10).to_string(index=False))

# Save failed parks info for debugging
failed_df = pd.DataFrame(failed_parks, columns=["Park", "Reason", "Details"]).head(200)
failed_df.to_csv(OUTPUT_DIR / "failed_parks_preview.csv", index=False)